In [1]:
import cv2
import numpy as np

In [2]:
#if u=you are y=using a external web cam the replace 0 by 1
cap = cv2.VideoCapture(0)

background = 0 #this will save background image for the program

for i in range(60):
    ret,background = cap.read()
    
#to get mirror effect we will flip background
background = np.flip(background , axis = 1)

while(True):
    ret,img = cap.read()
    
    #to get mirror effect we will flip img
    img = np.flip(img,axis=1)
    
    #converting our image to HSV (hue saturation value)
    
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    
    #detecting lower range of red
    lower_red = np.array([98,109,20])
    upper_red = np.array([112,255,255])
    mask_1 = cv2.inRange(hsv,lower_red,upper_red)
    
    #detecting higher range of red
    lower_red = np.array([240,109,20])
    upper_red = np.array([255,255,255])
    mask_2 = cv2.inRange(hsv,lower_red,upper_red)
    
    '''
    The Hue values are actually distributed over a circle (range between 0-360 degrees) 
    but in OpenCV to fit into 8bit value the range is from 0-180. The red color is represented by 0-30 
    as well as 150-180 values.

    We use the range 0-10 and 170-180 to avoid detection of skin as red. High range of 120-255 for
    saturation is used because our cloth should be of highly saturated red color.
    The lower range of value is 70 so that we can detect red color in the wrinkles of the cloth as well.

    '''
    
    #For generations of final mask
    #matrix addition
    mask_1 = mask_1+mask_2
    
    #Segmentation of red colour from mask
    mask_1 = cv2.morphologyEx(mask_1, cv2.MORPH_OPEN, np.ones((3,3),np.uint8))#to remove noise
    mask_1 = cv2.morphologyEx(mask_1, cv2.MORPH_DILATE, np.ones((3,3),np.uint8))#to dialate
    
    #in mask 1 we have a image that only shows red cloth
 
    #creating an inverted mask to segment out the cloth from the frame
    mask_2 = cv2.bitwise_not(mask_1) #if you have black it will give tou white
 
    #in mask 2 we have a image that shows Everything except cloth
 
    #Segmenting the cloth out of the frame using bitwise and with the inverted mask
    res1 = cv2.bitwise_and(img,img,mask=mask_2)
    
    # creating image showing static background frame pixels only for the masked region
    res2 = cv2.bitwise_and(background, background, mask = mask_1)
 
 
    #Generating the final output
    final_output = cv2.addWeighted(res1,1,res2,1,0)
    cv2.imshow("magic",final_output)
    
    #press esc for quiting
    if cv2.waitKey(10) == 27:
        break
        
cap.release()
cv2.destroyWindow('magic')
    